In [ ]:
import torch

if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    print(f"Find {gpu_count} GPU can be used.")

    # 遍历并打印每个GPU设备的名称
    for i in range(gpu_count):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i + 1}: {gpu_name}")
else:
    print("No GPU can be used.")


In [ ]:
import copy
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
# from bert_score import score
import statistics
from ast import literal_eval
import functools
import json
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

random.seed(8888)
torch.manual_seed(8888)
random.seed(8888)
np.random.seed(8888)

if torch.cuda.is_available():
    torch.cuda.manual_seed(8888)
    torch.cuda.manual_seed_all(8888)


from tqdm import tqdm

torch.set_grad_enabled(False)
tqdm.pandas()

In [ ]:
torch.cuda.set_device(0)


from transformers import AutoModelForCausalLM, AutoTokenizer
import json
from os.path import join

# Get CounterFact data for GPT2-xl, from the ROME repository.
#wget.download("https://rome.baulab.info/data/dsets/known_1000.json")


model_dir = "/mnt/workspace/workgroup/yhhong/transformers"
original_model_name = "Llama-2-7b-chat-hf" #"chatglm3-6b" #"Llama-2-7b-chat-hf" #"Qwen1.5-7B-Chat"


original_model = AutoModelForCausalLM.from_pretrained(
    join(model_dir, original_model_name),
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
);

tokenizer = AutoTokenizer.from_pretrained(join(model_dir, original_model_name))
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

original_model.to('cuda');


In [ ]:
original_model

In [ ]:
import torch.nn.functional as F

def set_act_get_hooks(model, mlp_coef=False, attn=False):
    """
    Works on LLaMA-2, getting coef values or attn values
    """

    for attr in ["activations_"]:
        if not hasattr(model, attr):
            setattr(model, attr, {})

    def get_activation(name):
        def hook(module, input, output):
            if "m_coef" in name:
                model.activations_[name] = input[0][:, :].detach() 
            elif "attn" in name:
                model.activations_[name] = output[:, :].detach() 

        return hook

    hooks = []
    for i in range(32):
        if mlp_coef is True: #co-effciency
            hooks.append(model.model.layers[i].mlp.down_proj.register_forward_hook(get_activation("m_coef_" + str(i))))
        if attn is True:
            hooks.append(model.model.layers[i].self_attn.o_proj.register_forward_hook(get_activation("attn_" + str(i))))    
        

    return hooks

def set_act_modify_hooks(model, original_model, mlp_coef, attn, ori_mlp_coef, ori_attn, un_mlp_coef, un_attn, layers=None):
    """
    Works on LLaMA-2, modifying coef values or attn values
    """

    def modify_activation(name, update_value, patch_input):
        def pre_hook(module, input):
            if "m_coef" in name:
                input[0][:, :, :] = update_value 
        
        def post_hook(module, input, output):
            if "attn" in name:
                output[:, :, :] = update_value
                
        if patch_input:
            return pre_hook
        else:
            return post_hook

    hooks = []
    if layers is not None:
        for i in range(32):
            if i in layers and mlp_coef is True: #co-effciency
                hooks.append(model.model.layers[i].mlp.down_proj.register_forward_pre_hook(modify_activation("m_coef_" + str(i), update_value = ori_mlp_coef[i], patch_input=True)))
            # else:
            #     hooks.append(model.model.layers[i].mlp.down_proj.register_forward_pre_hook(modify_activation("m_coef_" + str(i), update_value = un_mlp_coef[i], patch_input=True)))
            if i in layers and attn is True:
                hooks.append(model.model.layers[i].self_attn.o_proj.register_forward_hook(modify_activation("attn_" + str(i), update_value = ori_attn[i], patch_input=False)))  
            # else:
            #     hooks.append(model.model.layers[i].self_attn.o_proj.register_forward_hook(modify_activation("attn_" + str(i), update_value = un_attn[i], patch_input=False)))  

    return hooks

def remove_hooks(hooks):
    for hook in hooks:
        hook.remove()

        
def compute_loss(target_logits, logits):
    
    #MSE loss

    return torch.mean((target_logits - logits) ** 2)


def compute_KRS(ori_loss, new_loss):  
    return 1 - (new_loss / ori_loss)
    




In [ ]:
# loading data:

with open("data/llama2-7b_concepts_test_30tokens_answers.json", "r", encoding="utf-8") as file:
    data = json.load(file)
    
for ix, item in enumerate(data):
    print(ix," ",item['Concept'])



In [ ]:
# evaluate original model and obtain the next 30 tokens answers

for index, x in enumerate(tqdm(data)):
    
    questions = []
    n_new_tokens = 30
    for idx, question in enumerate(x['QA']):
        questions.append(f"Question: {question}\n Answer:")

    inputs = tokenizer(questions, return_tensors="pt", padding=True, return_token_type_ids=False).to('cuda')
    input_length = inputs.input_ids.size(1)
    with torch.no_grad():
        generation_output = original_model.generate(  # mt.model
            **inputs,
            do_sample=False,
            max_new_tokens=n_new_tokens,
        )

    
    eos_token_id = tokenizer.eos_token_id
    normal_tokens_num_list = []
    for i, output in enumerate(generation_output):
        eos_count = (output == eos_token_id).sum().item()
        total_tokens = output.size(0) 
        normal_tokens_count = total_tokens - eos_count - inputs.input_ids.size(1)
        normal_tokens_num_list.append(normal_tokens_count)
    
    outputs = tokenizer.batch_decode(generation_output[:, :], skip_special_tokens=True)
    
    data[index]['QA_with_answers'] = outputs
    data[index]['answers_tokens_num'] = normal_tokens_num_list

with open("data/llama2-7b_concepts_test_30tokens_answers.json", "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

In [ ]:
with open("data/llama2-7b_concepts_test_30tokens_answers.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [ ]:
#on vanilla model and unlearned model

from transformers import AutoModelForCausalLM, AutoTokenizer
import json
from os.path import join


model_dir = "/mnt/workspace/workgroup/yhhong/unlearn_results"

methods_ori_loss_list = [] 
methods_ori_attn_list = []
methods_ori_mlp_coef_list = []
methods_un_mlp_coef_list = []
methods_un_attn_list = []

for method in ['grad_diff', 'dpo']:
    ori_loss_list = [] 
    ori_attn_list = []
    ori_mlp_coef_list = []
    un_mlp_coef_list = []
    un_attn_list = []
    
    for x in tqdm(data):
        # Loading new unlearned model for certain concept 

        model_name = f"llama2-7b/{method}/Full/{x['id']}" 

        model = AutoModelForCausalLM.from_pretrained(
            join(model_dir, model_name),
            torch_dtype=torch.bfloat16,
            trust_remote_code=True
        );

        tokenizer = AutoTokenizer.from_pretrained(join(model_dir, model_name))
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "left"
        model.to('cuda')
        
        questions = x['QA_with_answers']

        inputs = tokenizer(questions, return_tensors="pt", padding=True, return_token_type_ids=False).to('cuda')
        input_length = inputs.attention_mask.sum(dim=1)

        un_hooks = set_act_get_hooks(model, mlp_coef=True, attn=True)
        with torch.no_grad():
            output = model(**inputs)

        # remove hooks
        remove_hooks(un_hooks) 

        hooks = set_act_get_hooks(original_model, mlp_coef=True, attn=True)
        with torch.no_grad():
            original_output = original_model(**inputs)
        # remove hooks
        remove_hooks(hooks) 
        
        
        loss_list = []
        for i, length in enumerate(x['answers_tokens_num']):
            loss = compute_loss(original_output.logits[i, -length:, :], output.logits[i, -length:, :])
            loss_list.append(loss)
        
        un_mlp_coef = []
        un_attn = []
        ori_mlp_coef = []
        ori_attn = []
        for layer in range(32):
            un_mlp_coef.append(model.activations_[f'm_coef_{layer}'])
            un_attn.append(model.activations_[f'attn_{layer}'])
            ori_mlp_coef.append(original_model.activations_[f'm_coef_{layer}'])
            ori_attn.append(original_model.activations_[f'attn_{layer}'])


        ori_loss_list.append(loss_list)

        un_mlp_coef_list.append(un_mlp_coef) # on all data
        un_attn_list.append(un_attn) # on all data

        ori_mlp_coef_list.append(ori_mlp_coef) # on all data
        ori_attn_list.append(ori_attn) # on all data


        del model
        del tokenizer
        torch.cuda.empty_cache()

    methods_ori_loss_list.append(ori_loss_list)
    methods_ori_attn_list.append(ori_attn_list)
    methods_ori_mlp_coef_list.append(ori_mlp_coef_list)
    methods_un_mlp_coef_list.append(un_mlp_coef_list)
    methods_un_attn_list.append(un_attn_list)
    



In [ ]:
#patching on MLP coefficients on unlearned models
torch.cuda.set_device(0)

from transformers import AutoModelForCausalLM, AutoTokenizer
import json
from os.path import join


model_dir = "/mnt/workspace/workgroup/yhhong/unlearn_results"


methods_mlp_coef_patch_loss_lists = []
for j, method in enumerate(['grad_diff', 'dpo']):
    mlp_coef_patch_loss_lists = []
    ori_mlp_coef_list = methods_ori_mlp_coef_list[j]
    ori_attn_list = methods_ori_attn_list[j]
    un_mlp_coef_list = methods_un_mlp_coef_list[j]
    un_attn_list = methods_un_attn_list[j]
    
    for index, x in enumerate(tqdm(data)):

        # Loading new unlearned model for certain concept 

        model_name = f"llama2-7b/{method}/Full/{x['id']}" #"chatglm3-6b" #"Llama-2-7b-chat-hf" #"Qwen1.5-7B-Chat"

        model = AutoModelForCausalLM.from_pretrained(
            join(model_dir, model_name),
            torch_dtype=torch.bfloat16,
            trust_remote_code=True
        );

        tokenizer = AutoTokenizer.from_pretrained(join(model_dir, model_name))
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "left"
        model.to('cuda');

        questions = x['QA_with_answers']
        inputs = tokenizer(questions, return_tensors="pt", padding=True, return_token_type_ids=False).to('cuda')
        input_length = inputs.attention_mask.sum(dim=1)

        with torch.no_grad():
            original_output = original_model(**inputs)

        patch_loss_list = []
        for layer in range(32):
            hooks = set_act_modify_hooks(model, original_model, mlp_coef=True, attn=False, ori_mlp_coef=ori_mlp_coef_list[index], ori_attn=ori_attn_list[index], un_mlp_coef=un_mlp_coef_list[index], un_attn=un_attn_list[index], layers = [layer-2, layer-1, layer, layer+1, layer+2])
            with torch.no_grad():
                output = model(**inputs)
            # remove hooks
            remove_hooks(hooks) 
            
            loss_list = []
            for i, length in enumerate(x['answers_tokens_num']):
                loss = compute_loss(original_output.logits[i, -length:, :], output.logits[i, -length:, :])
                loss_list.append(loss)
            
            patch_loss_list.append(loss_list)

        mlp_coef_patch_loss_lists.append(patch_loss_list)   


        del model
        del tokenizer
        torch.cuda.empty_cache()

    methods_mlp_coef_patch_loss_lists.append(mlp_coef_patch_loss_lists)



In [ ]:
print('recover on LLaMA by patching on coef (5 layer per patch)')
        
coef_KRS_list_all = []
for t, method in enumerate(['grad_diff', 'dpo']):
    print(f'on {method}')
    coef_KRS_list_per_method = []
    for patch_losses, ori_losses in zip(methods_mlp_coef_patch_loss_lists[t], methods_ori_loss_list[t]):
        coef_KRS_list_per_data = []
        for new_losses in patch_losses: #different_layer
            coef_KRS_list_per_layer = []
            for new_loss, ori_loss in zip(new_losses, ori_losses):
                KRS = compute_KRS(ori_loss, new_loss=new_loss)
                coef_KRS_list_per_layer.append(KRS.cpu())
    
            coef_KRS_list_per_data.append(np.mean(coef_KRS_list_per_layer, axis=-1))
        coef_KRS_list_per_method.append(coef_KRS_list_per_data)
     
    coef_KRS_list_per_method = np.array(coef_KRS_list_per_method)
    avg_coef_KRS_list_per_method = np.mean(coef_KRS_list_per_method, axis=0)
    
    coef_KRS_list_all.append(avg_coef_KRS_list_per_method)

coef_KRS_list_all = np.array(coef_KRS_list_all)
avg_coef_KRS_list_all = np.mean(coef_KRS_list_all, axis=0)
print(f'avg_coef_KRS_list_all: {list(avg_coef_KRS_list_all)}')

In [ ]:
#patching on Attention states on unlearned models

from transformers import AutoModelForCausalLM, AutoTokenizer
import json
from os.path import join


model_dir = "/mnt/workspace/workgroup/yhhong/unlearn_results"

methods_attn_patch_loss_lists = []
for j, method in enumerate(['grad_diff', 'dpo']):
    attn_patch_loss_lists = []
    ori_mlp_coef_list = methods_ori_mlp_coef_list[j]
    ori_attn_list = methods_ori_attn_list[j]
    un_mlp_coef_list = methods_un_mlp_coef_list[j]
    un_attn_list = methods_un_attn_list[j]
    
    for index, x in enumerate(tqdm(data)):

        # Loading new unlearned model for certain concept 

        model_name = f"llama2-7b/{method}/Full/{x['id']}" 

        model = AutoModelForCausalLM.from_pretrained(
            join(model_dir, model_name),
            torch_dtype=torch.bfloat16,
            trust_remote_code=True
        );

        tokenizer = AutoTokenizer.from_pretrained(join(model_dir, model_name))
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "left"
        model.to('cuda');

        questions = x['QA_with_answers']
        inputs = tokenizer(questions, return_tensors="pt", padding=True, return_token_type_ids=False).to('cuda')
        input_length = inputs.attention_mask.sum(dim=1)

        with torch.no_grad():
            original_output = original_model(**inputs)

        patch_loss_list = []
        for layer in range(32):
            hooks = set_act_modify_hooks(model, original_model, mlp_coef=False, attn=True, ori_mlp_coef=ori_mlp_coef_list[index], ori_attn=ori_attn_list[index], un_mlp_coef=un_mlp_coef_list[index], un_attn=un_attn_list[index], layers = [layer-2, layer-1, layer, layer+1, layer+2]) #[i for i in range(32)]
            with torch.no_grad():
                output = model(**inputs)
            # remove hooks
            remove_hooks(hooks) 
            
            loss_list = []
            for i, length in enumerate(x['answers_tokens_num']):
                loss = compute_loss(original_output.logits[i, -length:, :], output.logits[i, -length:, :])
                loss_list.append(loss)
            
            patch_loss_list.append(loss_list)

        attn_patch_loss_lists.append(patch_loss_list)   


        del model
        del tokenizer
        torch.cuda.empty_cache()

    methods_attn_patch_loss_lists.append(attn_patch_loss_lists)

In [ ]:
print('recover on LLaMA by patching on attn (5 layer per patch)')
        
attn_KRS_list_all = []
for t, method in enumerate(['grad_diff', 'dpo']):
    print(f'on {method}')
    attn_KRS_list_per_method = []
    for patch_losses, ori_losses in zip(methods_attn_patch_loss_lists[t], methods_ori_loss_list[t]):
        attn_KRS_list_per_data = []
        for new_losses in patch_losses: #different_layer
            attn_KRS_list_per_layer = []
            for new_loss, ori_loss in zip(new_losses, ori_losses):
                KRS = compute_KRS(ori_loss, new_loss=new_loss)
                attn_KRS_list_per_layer.append(KRS.cpu())
    
            attn_KRS_list_per_data.append(np.mean(attn_KRS_list_per_layer, axis=-1))
        attn_KRS_list_per_method.append(attn_KRS_list_per_data)
     
    attn_KRS_list_per_method = np.array(attn_KRS_list_per_method)
    avg_attn_KRS_list_per_method = np.mean(attn_KRS_list_per_method, axis=0)
    
    attn_KRS_list_all.append(avg_attn_KRS_list_per_method)

attn_KRS_list_all = np.array(attn_KRS_list_all)
avg_attn_KRS_list_all = np.mean(attn_KRS_list_all, axis=0)
print(f'avg_attn_KRS_list_all: {list(avg_attn_KRS_list_all)}')

In [ ]:
#patching on both MLP coef and Attention states on unlearned models

from transformers import AutoModelForCausalLM, AutoTokenizer
import json
from os.path import join


model_dir = "/mnt/workspace/workgroup/yhhong/unlearn_results"

methods_attn_mlp_coef_patch_loss_lists = []
for j, method in enumerate(['grad_diff', 'dpo']):
    attn_mlp_coef_patch_loss_lists = []
    ori_mlp_coef_list = methods_ori_mlp_coef_list[j]
    ori_attn_list = methods_ori_attn_list[j]
    un_mlp_coef_list = methods_un_mlp_coef_list[j]
    un_attn_list = methods_un_attn_list[j]

    for index, x in enumerate(tqdm(data)):

        model_name = f"llama2-7b/{method}/Full/{x['id']}" 

        model = AutoModelForCausalLM.from_pretrained(
            join(model_dir, model_name),
            torch_dtype=torch.bfloat16,
            trust_remote_code=True
        )

        tokenizer = AutoTokenizer.from_pretrained(join(model_dir, model_name))
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "left"
        model.to('cuda');

        questions = x['QA_with_answers']
        inputs = tokenizer(questions, return_tensors="pt", padding=True, return_token_type_ids=False).to('cuda')
        input_length = inputs.attention_mask.sum(dim=1)

        with torch.no_grad():
            original_output = original_model(**inputs)

        patch_loss_list = []
        for layer in range(32):
            hooks = set_act_modify_hooks(model, original_model, mlp_coef=True, attn=True, ori_mlp_coef=ori_mlp_coef_list[index], ori_attn=ori_attn_list[index], un_mlp_coef=un_mlp_coef_list[index], un_attn=un_attn_list[index], layers = [layer-2, layer-1, layer, layer+1, layer+2])
            with torch.no_grad():
                output = model(**inputs)
            # remove hooks
            remove_hooks(hooks) 
            
            loss_list = []
            for i, length in enumerate(x['answers_tokens_num']):
                loss = compute_loss(original_output.logits[i, -length:, :], output.logits[i, -length:, :])
                loss_list.append(loss)
            
            patch_loss_list.append(loss_list)

        attn_mlp_coef_patch_loss_lists.append(patch_loss_list)   


        del model
        del tokenizer
        torch.cuda.empty_cache()
    
    methods_attn_mlp_coef_patch_loss_lists.append(attn_mlp_coef_patch_loss_lists)



In [ ]:

print('recover on LLaMA by patching on both coef and attn (5 layer per patch)')
        
attn_mlp_coef_KRS_list_all = []
for t, method in enumerate(['grad_diff', 'dpo']):
    print(f'on {method}')
    attn_mlp_coef_KRS_list_per_method = []
    for patch_losses, ori_losses in zip(methods_attn_mlp_coef_patch_loss_lists[t], methods_ori_loss_list[t]):
        attn_mlp_coef_KRS_list_per_data = []
        for new_losses in patch_losses: #different_layer
            attn_mlp_coef_KRS_list_per_layer = []
            for new_loss, ori_loss in zip(new_losses, ori_losses):
                KRS = compute_KRS(ori_loss, new_loss=new_loss)
                attn_mlp_coef_KRS_list_per_layer.append(KRS.cpu())
    
            attn_mlp_coef_KRS_list_per_data.append(np.mean(attn_mlp_coef_KRS_list_per_layer, axis=-1))
        attn_mlp_coef_KRS_list_per_method.append(attn_mlp_coef_KRS_list_per_data)
     
    attn_mlp_coef_KRS_list_per_method = np.array(attn_mlp_coef_KRS_list_per_method)
    avg_attn_mlp_coef_KRS_list_per_method = np.mean(attn_mlp_coef_KRS_list_per_method, axis=0)
    
    attn_mlp_coef_KRS_list_all.append(avg_attn_mlp_coef_KRS_list_per_method)

attn_mlp_coef_KRS_list_all = np.array(attn_mlp_coef_KRS_list_all)
avg_attn_mlp_coef_KRS_list_all = np.mean(attn_mlp_coef_KRS_list_all, axis=0)
print(f'avg_attn_mlp_coef_KRS_list_all: {list(avg_attn_mlp_coef_KRS_list_all)}')

In [ ]:
#Restore MLP value vectors on unlearned models

from transformers import AutoModelForCausalLM, AutoTokenizer
import json
from os.path import join
import copy

global old_params


model_dir = "/mnt/workspace/workgroup/yhhong/unlearn_results"

methods_vectors_patch_loss_lists = []
for j, method in enumerate(['grad_diff', 'dpo']):
    vectors_patch_loss_lists = []
    ori_mlp_coef_list = methods_ori_mlp_coef_list[j]
    ori_attn_list = methods_ori_attn_list[j]
    un_mlp_coef_list = methods_un_mlp_coef_list[j]
    un_attn_list = methods_un_attn_list[j]

    for index, x in enumerate(tqdm(data)):

        model_name = f"llama2-7b/{method}/Full/{x['id']}" 

        model = AutoModelForCausalLM.from_pretrained(
            join(model_dir, model_name),
            torch_dtype=torch.bfloat16,
            trust_remote_code=True
        )

        old_params = copy.deepcopy(model.state_dict())

        tokenizer = AutoTokenizer.from_pretrained(join(model_dir, model_name))
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "left"
        model.to('cuda')

        questions = x['QA_with_answers']
        inputs = tokenizer(questions, return_tensors="pt", padding=True, return_token_type_ids=False).to('cuda')
        input_length = inputs.attention_mask.sum(dim=1)

        with torch.no_grad():
            original_output = original_model(**inputs)

        vectors_patch_loss_list = []
        for layer in range(32):
            for recover_layer in range(32): # select which layer's MLP value vectors to be recover:
                if recover_layer >= layer -2 and recover_layer <= layer + 2:
                    model.state_dict()[f'model.layers.{recover_layer}.mlp.down_proj.weight'][:, :] = original_model.state_dict()[f'model.layers.{recover_layer}.mlp.down_proj.weight'][:, :]
            
            
            with torch.no_grad():
                output = model(**inputs)

            loss_list = []
            for i, length in enumerate(x['answers_tokens_num']):
                loss = compute_loss(original_output.logits[i, -length:, :], output.logits[i, -length:, :])
                loss_list.append(loss)

            vectors_patch_loss_list.append(loss_list)

            model.load_state_dict(old_params)
            
        vectors_patch_loss_lists.append(vectors_patch_loss_list)   

        del model
        del tokenizer
        torch.cuda.empty_cache()
        
    methods_vectors_patch_loss_lists.append(vectors_patch_loss_lists)
    



In [ ]:


print('recover on LLaMA by patching on value vectors (5 layer per patch)')
        
vectors_KRS_list_all = []
for t, method in enumerate(['grad_diff', 'dpo']):
    print(f'on {method}')
    vectors_KRS_list_per_method = []
    for patch_losses, ori_losses in zip(methods_vectors_patch_loss_lists[t], methods_ori_loss_list[t]):
        vectors_KRS_list_per_data = []
        for new_losses in patch_losses: #different_layer
            vectors_KRS_list_per_layer = []
            for new_loss, ori_loss in zip(new_losses, ori_losses):
                KRS = compute_KRS(ori_loss, new_loss=new_loss)
                vectors_KRS_list_per_layer.append(KRS.cpu())
    
            vectors_KRS_list_per_data.append(np.mean(vectors_KRS_list_per_layer, axis=-1))
        vectors_KRS_list_per_method.append(vectors_KRS_list_per_data)
     
    vectors_KRS_list_per_method = np.array(vectors_KRS_list_per_method)
    avg_vectors_KRS_list_per_method = np.mean(vectors_KRS_list_per_method, axis=0)
    
    vectors_KRS_list_all.append(avg_vectors_KRS_list_per_method)

vectors_KRS_list_all = np.array(vectors_KRS_list_all)
avg_vectors_KRS_list_all = np.mean(vectors_KRS_list_all, axis=0)
print(f'avg_vectors_KRS_list_all: {list(avg_vectors_KRS_list_all)}')